### Travelling Salesman Problem

The proverbial seller visits $N$ (assuming here 20 by default) cities with given positions $(x_i, y_i)$, returning finally the city of origin. Each city is to be visited only once, and the route is to be made as short as possible.

In [3]:
from math import sqrt, floor, exp
import collections.abc as abc
import secrets
from copy import deepcopy as cp

import numpy as np
from numpy.lib.recfunctions import unstructured_to_structured as unstruc_to_struc

import graphviz as gv
import imageio.v2 as imageio

In [4]:
# to reproduce
seed = 203089051535183421231184365759039973676
# seed = secrets.randbits(128)
rng = np.random.default_rng(seed)

def reset_rng():
  global rng
  rng = np.random.default_rng(seed)

In [47]:
class Graph:  
  def __init__(self, n_nodes: int = 20):
    # generate random node position
    self.n_nodes = n_nodes
    self.nodes = np.array([])
    while len(self.nodes) < self.n_nodes:
      self.nodes = rng.integers(low=0., high=100., size=(self.n_nodes, 2))
      self.nodes = unstruc_to_struc(self.nodes, dtype=np.dtype([('x', int), ('y', int)]))
      self.nodes = np.unique(np.sort(self.nodes, order=['x', 'y']))
    

    self.dist = np.zeros(shape=(self.n_nodes, self.n_nodes))
    for r in range(self.n_nodes):
      for c in range(r):
        self.dist[r][c] = self.dist[c][r]
      for c in range(r + 1, self.n_nodes):
        self.dist[r][c] = sqrt(
          (self.nodes[r]['x'] - self.nodes[c]['x']) ** 2 +
          (self.nodes[r]['y'] - self.nodes[c]['y']) ** 2
        )
  
  
  def distance(self, node1: int, node2: int) -> float:
    return self.dist[node1][node2]
    
    
  def route_len(self, route: abc.Sequence[int]) -> float:
    rlen = 0
    for i in range(1, self.n_nodes):
      rlen += self.dist[route[i]][route[i - 1]]
    rlen += self.dist[route[-1]][route[0]]
    return rlen
  
  
  # saves graph with the given route to ./route/route_{idx}.png
  def visualize(self, route: abc.Sequence[int], idx: int = 0):
    vis = gv.Graph('salesman_route', engine='neato', format='png')
    vis.attr(label=f'gen: {idx}, len: {self.route_len(route)}')
    
    for i in range(self.n_nodes):
      pos = f"{self.nodes[i]['x'] / 15},{self.nodes[i]['y'] / 15}!"
      vis.node(str(i), pos=pos, width='0', height='0')

    for i in range(1, self.n_nodes):
      vis.edge(str(route[i]), str(route[i - 1]), color='#ac4242')
    vis.edge(str(route[self.n_nodes - 1]), str(route[0]), color='#ac4242')

    vis.render(cleanup=True, directory='route', filename=f'route_{idx}')

In [48]:
def route_evolution_gif(evolution_depth: int, gifname_suffix: str = ''):
  images = []
  for idx in range(evolution_depth):  
    images.append(imageio.imread(f'route/route_{idx}.png'))
  imageio.mimsave(f'route_evolution{gifname_suffix}.gif', images)

#### Ants Colony Optimization

The algorithm simulates the behavior of ants. In nature, ants leave pheromone trails. The ant wanders randomly, unless it comes across a pheromone trail.In that case, it is more likely that it will follow the trail. The more pheromones there are, the more ants have passed through here, meaning this route is good enough.

The algorithm creates several generations (their number is controlled by the `n_gener` parameter) of ant colonies, one after the other. In each generation, except for the first, the ants rely on pheromones left behind by previous generations. From every node, several ants leave (`n_ants` parameter) in every generation. The ants in each node select the next node randomly with respect to the pheromone signals on the edges. Additionally, there are elite ants (`n_elites` parameter) who only follow the best routes (i.e., those with the most pheromone signals). First generation starts with equal pheromones on each edge (`pher_init`). From generation to generation, pheromones evaporate with a certain coefficient (`vapor_cf`). 

**Probability of ant moving from i-th to j-th node:**
$ P_{ij}=\cfrac{\tau_{ij}^\alpha\eta_{ij}^\beta}{\sum{\tau_{im}^\alpha\eta_{im}^\beta}} $

$\tau - \text{pheromones}$, 
$\eta - \text{inversed distance}$, 
$\alpha,\beta - \text{constant parameters}$


In [49]:
def ants_colony(graph: Graph, n_gener=50, n_ants=4, n_elites=0.5,
                alfa=1, beta=3, pher_delta_cf=3, vapor_cf=0.5, visualize=True) \
    -> abc.Sequence[int]:
  
  PHER_INI = 0.5
  PHER_MAX = 1.0

  # pheromones for the first generation, all equals
  pheromone = np.full((graph.n_nodes, graph.n_nodes), PHER_INI)
  # the matrix of values inversed to the distance with certain coefficient
  closeness = np.zeros(shape=(graph.n_nodes, graph.n_nodes))
  for r in range(graph.n_nodes):
    for c in range(r):
      closeness[r][c] = closeness[c][r]
    for c in range(r + 1, graph.n_nodes):
      closeness[r][c] = 1 / graph.distance(r, c)
  
  # each ant's route and the best (i.e. the shortest) route found so far
  # route end connected to the beginning so it's cycle
  curr_route = list(range(graph.n_nodes))
  best_route = list(range(graph.n_nodes))
  # best route length
  best_len = float('inf')
  
  # just a list of node indices to pass it to numpy.random.choice
  next_pos_choices = list(range(graph.n_nodes))
  
  # best_route becomes better through generations
  for gener in range(n_gener):
    # matrix of newly laid pheromones by current generation
    # pheromones are recomputed at the end of the current generation
    pher_delta = np.zeros(shape=(graph.n_nodes, graph.n_nodes))
    
    # n_ants leaves from each node
    for ant in range(n_ants * graph.n_nodes):
      pos = ant % graph.n_nodes
      
      # controlling current route
      curr_route[0] = pos
      curr_idx = 1
      curr_len = 0
      
      # ant marks visited nodes with truthy values
      visited = np.array([False] * graph.n_nodes)
      
      # iterate through ant steps (their number is n_nodes - 1)
      # the last step is determined by the last and the first nodes in the route
      for step in range(1, graph.n_nodes):
        # mark current node as visited
        visited[pos] = True
        # probablity of moving to each node from the current one
        mv_prob = np.zeros(graph.n_nodes)
        mv_prob_sum = 0
        
        # iterating through all the nodes
        for dest in range(graph.n_nodes):
          # ant won't move to the node where it have already been
          if visited[dest]:
            mv_prob[dest] = 0
            continue
          
          # probability of moving this node
          mv_prob[dest] = (pheromone[pos][dest] ** alfa) * (closeness[pos][dest] ** beta)
          mv_prob_sum += mv_prob[dest]
        
        # to make probabilities sum up to 1
        mv_prob /= mv_prob_sum

        # choose next node randomly with respect to probabilities
        next_pos = np.random.choice(next_pos_choices, p=mv_prob)
        
        # updating current route
        curr_route[curr_idx] = next_pos
        curr_idx += 1
        curr_len += graph.distance(pos, next_pos)
        pos = next_pos
      
      # add the edge from the last to the first node
      curr_len += graph.distance(curr_route[-1], curr_route[0])
      
      # pheromones signals laid by the current ant
      delta = pher_delta_cf / curr_len
      for step in range(1, graph.n_nodes):
        pher_delta[curr_route[step - 1]][curr_route[step]] += delta
      pher_delta[curr_route[-1]][curr_route[0]] += delta
      
      # updating best route
      if curr_len < best_len:
        best_route, curr_route = curr_route, best_route
        best_len = curr_len
        
    # elite ants go only on the best route and lay pheromones there
    elite_delta = floor(n_elites * graph.n_nodes) * pher_delta_cf / best_len
    for step in range(1, graph.n_nodes):
      pher_delta[best_route[step - 1]][best_route[step]] += elite_delta
    pher_delta[best_route[-1]][best_route[0]] += elite_delta
  
    # updating pheromones signals
    for r in range(graph.n_nodes):
      for c in range(graph.n_nodes):
        pheromone[r][c] = min(vapor_cf * pheromone[r][c] + pher_delta[r][c], PHER_MAX)
    
    if visualize:
      graph.visualize(best_route, gener)

  return best_route

In [50]:
def hyperparam_optim():
  # hyperparameter optimization
  choices = {
    'n_ants': [1, 2, 3, 4],
    'n_elites': [0.3, 0.5, 0.7, 1],
    'alfa': [0.3, 0.5, 0.8, 1, 2, 3, 5, 7, 9],
    'beta': [0.3, 0.5, 0.8, 1, 2, 3, 5, 7, 9],
    'close_cf': [0.3, 0.5, 0.8, 1, 2, 3, 5, 7, 9],
    'vapor_cf': [0.2, 0.3, 0.5, 0.7, 0.8],
    'pher_delta_cf': [0.5, 0.8, 1, 2, 3, 5, 7],
  }

  # number of graph nodes
  n_nodes = 30
  # number of random test graphs
  n_graphs = 30
  # number of random param configurations 
  n_params = 20

  # random test graphs
  graphs = [Graph(rng, n_nodes) for i in range(n_graphs)]
  # random param configurations
  params = [{k: rng.choice(choices[k]) for k in choices} for i in range(n_params)]
  # for each graph algo with the given parameters finds shortest route
  # for each graph all the parameters
  total = [0] * n_params

  for graph in graphs:
    for i, param in enumerate(params):
      total[i] += graph.route_len(ants_colony(graph, n_gener=50, visualize=False, **param))
       
  return total, params

#### Simulated annealing technique

At the heart of the method of simulated annealing is an analogy with thermodynamics, specifically with the way that liquids freeze and crystallize, or metals cool and anneal. At high temperatures, the molecules of a liquid move freely with respect to one another. If the liquid is cooled slowly, thermal mobility is lost. The atoms are often able to line themselves up and form a pure crystal that is completely ordered over a distance up to billions of times the size of an individual atom in all directions. This crystal is the state of minimum energy for this system. The amazing fact is that, for slowly cooled systems, nature is able to find this minimum energy state. In fact, if a liquid metal is cooled quickly or “quenched,” it does not reach this state but rather ends up in a polycrystalline or amorphous state having somewhat higher energy.

So the essence of the process is slow cooling, allowing ample time for redistribution of the atoms as they lose mobility. This is the technical definition of annealing, and it is essential for ensuring that a low energy state will be achieved.

Even at low temperature, there is a chance, albeit very small, of a system being in a high energy state. In other words, the system sometimes goes uphill as well as downhill; but the lower the temperature, the less likely is any significant uphill excursion. **A simulated thermodynamic system is assumed to change its configuration from energy $E_1$ to energy $E_2$ with probability** $p = \exp{\cfrac{-(E_2 - E_1)}{kT}}$. The quantity $k$ (Boltzmann’s constant) is a constant of nature that relates temperature to energy.

In [51]:
# issues a verdict on whether to accept reconfiguration
# that leads to the given energy change when the system has given temperature
def consult_oracle(delta_ener, tempr):
  return delta_ener < 0 or rng.random() < exp(-delta_ener / tempr)


# delta of energy (i.e. delta of the change in the curr_len) in case of
# segment [0:segment_len] transportation such that it follows curr_route[segment_len + head]
def delta_ener_transport(graph: Graph, curr_route: np.ndarray, segment_len: int, head: int):
  de = 0
  
  # cut segment
  de -= graph.distance(curr_route[-1], curr_route[0])
  de -= graph.distance(curr_route[segment_len - 1], curr_route[segment_len])
  de += graph.distance(curr_route[-1], curr_route[segment_len])
  
  # paste segment
  de -= graph.distance(curr_route[segment_len + head], curr_route[segment_len + head + 1])
  de += graph.distance(curr_route[segment_len + head], curr_route[0])
  de += graph.distance(curr_route[segment_len - 1], curr_route[segment_len + head + 1])
  
  return de


# transport segment [0:segment_len] such that it follows curr_route[segment_len + head]
def do_transport(curr_route: np.ndarray, segment_len: int, head: int):
  curr_route = np.concatenate((
    curr_route[segment_len:segment_len + head + 1],
    curr_route[:segment_len],
    curr_route[segment_len + head + 1:]  
  ))


# delta of energy (i.e. delta of the change in the curr_len) in case of
# segment [0:segment_len] reversal
def delta_ener_reversal(graph: Graph, curr_route: list[int], segment_len: int):
  de = 0
  
  # cut segment
  de -= graph.distance(curr_route[-1], curr_route[0])
  de -= graph.distance(curr_route[segment_len - 1], curr_route[segment_len])
  
  # paste segment
  de += graph.distance(curr_route[0], curr_route[segment_len])
  de += graph.distance(curr_route[segment_len - 1], curr_route[-1])
  
  return de


# reverses segment [0:segment_len]
def do_reversal(curr_route: list[int], segment_len: int):
  curr_route[:segment_len] = curr_route[:segment_len][::-1]


def simulated_annealing(graph: Graph, cool_cf=0.9, max_tempr_steps=100,
                        max_reconf_tried=100, max_reconf_done=10, visualize=True) \
    -> abc.Sequence[int]:
  
  # curr_route is represented by sequence of indices of nodes in order of visiting
  # starting with random curr_route
  curr_route = np.array(range(graph.n_nodes))
  rng.shuffle(curr_route)
  best_route = cp(curr_route)
  # current route curr_len = current system energy
  curr_len = graph.route_len(curr_route)
  best_len = float('inf')
  
  # system temperature controls probability of increasing energy (i.e. curr_route prolonging) 
  tempr = 1
  
  # how many times temperature decreases (i.e. multiplied by cool_cf < 1)
  for tempr_step in range(max_tempr_steps):
    
    # how many times reconfiguration tried / done (done after consulting the oracle)
    reconf_tried = 0
    reconf_done = 0
    
    # do reconfigurations
    while reconf_tried < max_reconf_tried and reconf_done < max_reconf_done:
      # segment of the current route to be reconfigured
      segment_start = rng.integers(1, graph.n_nodes)
      segment_len = rng.integers(2, graph.n_nodes - 2)
      curr_route = np.concatenate((curr_route[segment_start:], curr_route[:segment_start]))
  
      # two types of current route reconfigurations:
      # 1. reverse segment
      # 2. transport segment to another place in the current route
      # whether to do segment reversal or transport
      reversal_chosen = rng.choice([True, False])
      
      # current route reconfiguration
      reconf_tried += 1
      if reversal_chosen:
        de = delta_ener_reversal(graph, curr_route, segment_len)
        # accept next current route if it's shorter or oracle says that
        if consult_oracle(de, tempr):
          reconf_done += 1
          do_reversal(curr_route, segment_len)
          curr_len += de
      # transport chosen
      else:
        head = rng.integers(0, graph.n_nodes - segment_len - 1)
        de = delta_ener_transport(graph, curr_route, segment_len, head)
        # accept next current route if it's shorter or oracle says that
        if consult_oracle(de, tempr):
          reconf_done += 1
          do_transport(curr_route, segment_len, head)
          curr_len += de
        
      if curr_len < best_len:
        best_route[:] = curr_route[:]
        best_len = curr_len
         
    if visualize:
      graph.visualize(best_route, tempr_step)
      
    # temperature decreses
    tempr *= cool_cf

In [52]:
reset_rng()
n_nodes = 20
graph = Graph(n_nodes)

In [53]:
n_gener = 50
ants_colony(graph, n_gener=n_gener)
route_evolution_gif(evolution_depth=n_gener, gifname_suffix='_ants')

In [46]:
max_tempr_steps = 100
simulated_annealing(graph, max_tempr_steps=max_tempr_steps)
route_evolution_gif(evolution_depth=max_tempr_steps, gifname_suffix='_annealing')